# CS 195: Natural Language Processing
## Embeddings

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F5_4_Embeddings.ipynb)


## References

Word2Vec Tutorial - The Skip-Gram Model by Chris McCormick: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

Word2Vec - Negative Sampling made easy by Munesh Lakhey: https://medium.com/@mnshonco/word2vec-negative-sampling-made-easy-9a587cb4695f

Keras Embedding Layer: https://keras.io/api/layers/core_layers/embedding/

Keras Tokenizer: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow

## Dataset for today

AG News dataset
* short news articles
* four classes: World, Sports, Business, Sci/Tech

https://huggingface.co/datasets/ag_news


In [25]:
from datasets import load_dataset
data = load_dataset("ag_news")

print(data["train"]["text"][0])

# 0 is World
# 1 is Sports
# 2 is Business
# 3 is Sci/Tech
print(data["train"]["label"][0])


Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2


## Categorical classification example (with > 2 classes) 

We've only seen binary classification examples so far
* used binary_crossentropy loss and a sigmoid output unit

For more than two classes, use categorical_crossentropy and softmax

In [38]:
from datasets import load_dataset
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")
print("Here's an example of a sentence from the dataset:",data["train"]["text"][0])

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])
print("Here's an example of a tokenized sentence converted into a sequence of integers:",train_sequences[0])

# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')
print("Here's an example after it has been padded:",train_sequences[0])


# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)
print("Here's an example of what the target label looks like:", train_labels_array[0])


#create a neural network architecture
model = Sequential()
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")

Here's an example of a sentence from the dataset: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Here's an example of a tokenized sentence converted into a sequence of integers: [442, 441, 1681, 14528, 108, 64, 1, 850, 21, 21, 753, 8196, 442, 6640, 10231, 2927, 4, 5810, 25989, 40, 4049, 797, 332]
Here's an example after it has been padded: [442, 441, 1681, 14528, 108, 64, 1, 850, 21, 21, 753, 8196, 442, 6640, 10231, 2927, 4, 5810, 25989, 40, 4049, 797, 332]
Here's an example of what the target label looks like: [0. 0. 1. 0.]
Epoch 1/10
3750/3750 [==============================] - 2s 562us/step - loss: 139.1792 - accuracy: 0.2525
Epoch 2/10
3750/3750 [==============================] - 2s 546us/step - loss: 1.6659 - accuracy: 0.2538
Epoch 3/10
3750/3750 [==============================] - 2s 505us/step - loss: 1.3985 - accuracy: 0.2507
Epoch 4/10
3750/3750 [==============================] - 2

## Why does this do so badly?

No matter how many epochs you run this for, it will not get any better

**The problem:** the integer encoding

So what do we do if we still want to feed *sequential* data - we don't just want a bag of words?

## Word Embeddings

Don't just represent words with a single number - use a whole vector

<div>
   <img src="images/embeddings.png"> 
</div>

image source: https://stackoverflow.com/questions/46155868/keras-embedding-layer

in reality, you can't label the dimensions with exact meanings like "living being", "feline", etc.
* but you don't need to!

## How do we come up with these embeddings?

First, we'll come up with a "fake" learning problem and then extract information about words from the model

**fake learning problem** skip-grams: predict the words that appear around a given word

We can pick any window size - this one uses size 2 (2 words before and 2 after)


<div>
   <img src="images/skip_gram_problem.png"> 
</div>

image source: http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

## One Hot Encoding

Before we can create a model for this, we need an initial numerical encoding of words

**One Hot Encoding** uses a vector with length equal to the size of the vocabulary
* 1 in the spot representing that word
* 0 in all others

|            | the | quick | brown | fox | jumps | over | lazy | dog |
|------------|-----|-----|-----|----|-----|-----|-----|-----|
| "fox" | 0   | 0   | 0   | 1  | 0   | 0   | 0   | 0   |
| "dog" | 0 | 0   | 0   | 0   | 0  | 0   | 0   | 1   | 

We can use Keras' `to_categorical` for this

In [26]:
from keras.utils import to_categorical

#put a 1 at index 3 of 8
print( to_categorical(3, num_classes=8)  )

[0. 0. 0. 1. 0. 0. 0. 0.]


## Let's get started

Here are some toy sentences we can work with.

We'll use Keras' tokenizer and show the mapping of words to indexes it cam up with

In [30]:
from keras.preprocessing.text import Tokenizer

# Sample data
sentences = [
    "I adopted some dogs from the animal shelter",
    "don't you know that dogs and cats both like scritches",
    "are cats or dogs your favorite animal",
    "I have heard that dogs can be obedient",
    "I have heard that cats can be independent",
    "sharks live in the ocean",
    "many birds fly to get around"
]

# Tokenize and create vocabulary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)


{'dogs': 1, 'i': 2, 'that': 3, 'cats': 4, 'the': 5, 'animal': 6, 'have': 7, 'heard': 8, 'can': 9, 'be': 10, 'adopted': 11, 'some': 12, 'from': 13, 'shelter': 14, "don't": 15, 'you': 16, 'know': 17, 'and': 18, 'both': 19, 'like': 20, 'scritches': 21, 'are': 22, 'or': 23, 'your': 24, 'favorite': 25, 'obedient': 26, 'independent': 27, 'sharks': 28, 'live': 29, 'in': 30, 'ocean': 31, 'many': 32, 'birds': 33, 'fly': 34, 'to': 35, 'get': 36, 'around': 37}


### So now we know all the words in our vocabulary

0 will be a special index to represent "None"

In [28]:
vocabulary_size = len(tokenizer.word_index) + 1 #we also have to account for 0
print(vocabulary_size)

38


### Now we tokenize the sentences and convert them to their indexes

In [29]:
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[2, 11, 12, 1, 13, 5, 6, 14], [15, 16, 17, 3, 1, 18, 4, 19, 20, 21], [22, 4, 23, 1, 24, 25, 6], [2, 7, 8, 3, 1, 9, 10, 26], [2, 7, 8, 3, 4, 9, 10, 27], [28, 29, 30, 5, 31], [32, 33, 34, 35, 36, 37]]


### Creating the skip grams

Keras has a nice function for this too.

In [31]:
from keras.preprocessing.sequence import skipgrams

sequence_skipgrams = skipgrams(sequences[0],vocabulary_size=vocabulary_size,window_size=3)
print(sequence_skipgrams)

([[13, 14], [5, 19], [13, 19], [1, 29], [5, 14], [11, 14], [11, 13], [11, 19], [12, 5], [5, 6], [2, 9], [12, 10], [1, 11], [6, 5], [1, 13], [13, 5], [5, 1], [1, 14], [13, 36], [5, 13], [5, 5], [11, 2], [5, 14], [12, 28], [13, 3], [14, 6], [14, 13], [6, 14], [13, 1], [6, 13], [2, 11], [2, 12], [13, 34], [2, 14], [12, 13], [14, 5], [11, 12], [5, 13], [6, 1], [14, 6], [14, 27], [12, 21], [11, 5], [12, 2], [11, 1], [5, 18], [1, 12], [1, 23], [6, 12], [14, 23], [12, 1], [11, 23], [12, 10], [5, 12], [1, 12], [1, 18], [12, 34], [2, 30], [6, 24], [12, 11], [1, 5], [2, 1], [6, 11], [13, 12], [1, 2], [1, 5], [13, 22], [13, 11], [13, 6], [1, 6], [13, 5], [6, 29]], [1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0])


### Group Discussion

What does this output mean?

What are the different pairs of numbers?

What are all the 0s and 1s at the end?

The `skipgrams` function is doing something called **negative sampling**. Write your guess for what that means here.

**Negative sampling:**

### Group Exercise: Prepare data for the skip-gram model

use `sequence_skipgrams` to prepare the data

Each training example should be the one hot encoded word concatenated with the one hot encoded context word

Example: if "brown" is index 3 and "fox" is index 4 (and vocab size is 8), then we have

"brown":`[0,0,0,1,0,0,0,0]`

"fox":`[0,0,0,0,1,0,0,0]`

training example input: `[0,0,0,1,0,0,0,0, 0,0,0,0,1,0,0,0]`

*Hint:* you will need to use the `to_categorical` function

Then, do it for all the skipgrams

## A simple model

Let's draw what this model looks like on the white board

Then we'll train it with the data we prepared

In [33]:
from keras.models import Sequential
from keras.layers import Dense

# Model
embedding_model = Sequential()
# we input a one-hot vector for the word and its context word, so vocabulary_size*2
embedding_model.add(Dense(50, input_dim=vocabulary_size*2, activation='relu')) 
embedding_model.add(Dense(1, activation='sigmoid'))

embedding_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

embedding_model.fit(inputs_array, labels_array, epochs=5000, verbose=0)


## Where is the word embedding in all this?

The weights going from the word's index to hidden layer nodes represent what the model learned about this word, so we'll use those weights as the embedding

In [35]:
word = "dogs"
print("Word:", word)
word_index = tokenizer.word_index[word]
print("index:",word_index)
weights = embedding_model.layers[0].get_weights()[0]
word_embedding = weights[word_index]
print(f"The embedding for the word '{word}' is: {word_embedding.flatten()}")

Word: dogs
index: 1
The embedding for the word 'dogs' is: [-0.05164365  0.15345961  0.27586758  0.03484106  0.01857457  0.12105501
 -0.12195754  0.3844508   0.52245075 -0.35440248 -1.3468415  -0.35215762
 -0.18128224  0.342001    0.29873767 -0.80867183  0.37532702 -0.06673234
  0.36035883  0.27477598  0.04212696  0.46738985 -0.17842224 -0.08797641
 -0.12868786  0.07381967 -0.10810392  0.09558013 -0.13766237 -0.35415867
  0.02245924  0.21492355  0.39072448  0.2834317  -0.28319678 -0.35475025
  0.21243913 -0.47210756  0.27767736  0.27440387  0.04292269 -0.08132959
 -0.04046155  0.0301704  -0.25367644 -0.05824045 -1.1348718  -0.4878483
  0.13669956  0.25040466]


### We could make this into a function

In [36]:
def get_embedding(word,embedding_model):
    word_index = tokenizer.word_index[word]
    weights = embedding_model.layers[0].get_weights()[0]
    word_embedding = weights[word_index]
    return word_embedding

cats_embedding = get_embedding("cats",embedding_model)
dogs_embedding = get_embedding("dogs",embedding_model)
shelter_embedding = get_embedding("shelter",embedding_model)
sharks_embedding = get_embedding("sharks",embedding_model)
print(cats_embedding)
print(dogs_embedding)

print( np.sum(np.square(cats_embedding-dogs_embedding)) )
print( np.sum(np.square(sharks_embedding-shelter_embedding)) )

[-0.02375348  0.11949711  0.06044994 -0.03714155 -0.19955637 -0.07614389
 -0.05473538  0.02902077  0.18962364 -0.21753238  0.16662757  0.10069387
  0.17979206 -0.16885345  0.01473857 -0.0690915   0.13704668  0.06257458
  0.17304529  0.10747083 -0.026731   -0.05845037 -0.16764595  0.02230105
 -0.17020188 -0.08070949  0.03694488  0.21064727 -0.14492504  0.14048947
 -0.21059206  0.10628273 -0.05432342 -0.00303474 -0.14579701  0.01792982
 -0.03013816  0.04185806  0.20681618 -0.14459702  0.0063161  -0.1687189
 -0.0495749   0.0276676   0.1956714  -0.05057637 -0.10117345  0.03816085
  0.18405418  0.10212852]
[-0.05164365  0.15345961  0.27586758  0.03484106  0.01857457  0.12105501
 -0.12195754  0.3844508   0.52245075 -0.35440248 -1.3468415  -0.35215762
 -0.18128224  0.342001    0.29873767 -0.80867183  0.37532702 -0.06673234
  0.36035883  0.27477598  0.04212696  0.46738985 -0.17842224 -0.08797641
 -0.12868786  0.07381967 -0.10810392  0.09558013 -0.13766237 -0.35415867
  0.02245924  0.21492355  

## Applied Exploration

Create word embeddings for the AG News dataset.

Put all of the code into one cell so it isn't spread all throughout the notebook.

Show some example word embeddings.

Describe your results and reflect on them
* How many unique words does the dataset have?
* How many training epochs do you think are appropriate? Why?
* How could you go about figuring out if these embeddings are useful?



## The Keras Embedding Layer

Keras provides an `Embedding` layer that you can put at the beginning of your network which allows you to learn the embeddings as part of the main training process.

Let's see how it does when we include this in our example from the beginning of this notebook.

In [37]:
from datasets import load_dataset
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from datasets import load_dataset
import numpy as np


data = load_dataset("ag_news")

# Prepare the tokenizer and fit on the training text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["train"]["text"])
vocabulary_size = len(tokenizer.word_index) + 1

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(data["train"]["text"])
test_sequences = tokenizer.texts_to_sequences(data["test"]["text"])



# Pad sequences to ensure uniform length; you can decide the max length based on your dataset's characteristics
max_length = 100  # This should be adjusted based on the dataset
train_encoding_array = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_encoding_array = pad_sequences(test_sequences, maxlen=max_length, padding='post')


# Convert labels to one-hot vectors
train_labels = data["train"]["label"]
test_labels = data["test"]["label"]
train_labels_array = to_categorical(train_labels, num_classes=4)
test_labels_array = to_categorical(test_labels, num_classes=4)

#create a neural network architecture
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=50, input_length=max_length))
model.add(Flatten())
model.add(Dense(20, input_dim=max_length, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_encoding_array, train_labels_array, epochs=10, verbose=1)

loss, accuracy = model.evaluate(test_encoding_array, test_labels_array)
print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.3233 - accuracy: 0.8827
Epoch 2/10
3750/3750 [==============================] - 26s 7ms/step - loss: 0.1125 - accuracy: 0.9632
Epoch 3/10
3750/3750 [==============================] - 27s 7ms/step - loss: 0.0313 - accuracy: 0.9914
Epoch 4/10
3750/3750 [==============================] - 28s 7ms/step - loss: 0.0175 - accuracy: 0.9963
Epoch 5/10
3750/3750 [==============================] - 28s 7ms/step - loss: 0.0123 - accuracy: 0.9973
Epoch 6/10
3750/3750 [==============================] - 27s 7ms/step - loss: 0.0074 - accuracy: 0.9980
Epoch 7/10
3750/3750 [==============================] - 28s 8ms/step - loss: 0.0062 - accuracy: 0.9981
Epoch 8/10
3750/3750 [==============================] - 27s 7ms/step - loss: 0.0043 - accuracy: 0.9983
Epoch 9/10
3750/3750 [==============================] - 29s 8ms/step - loss: 0.0034 - accuracy: 0.9987
Epoch 10/10
238/238 [==============================] - 0s 628us/step - lo